# Task 1
## Install beautifulsoup4 and import libraries

In [2]:
#pip install beautifulsoup4    #Comment out after installation successful
import urllib.request, urllib.parse, urllib.error
import ssl
from bs4 import BeautifulSoup

In [3]:
import numpy as np
import pandas as pd

## Read data from Web into a dataframe

In [4]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urllib.request.urlopen(url, context=ctx).read()
soup = BeautifulSoup(html, 'html.parser')

table = soup.find_all('table')[0] # Grab the first table

df = pd.DataFrame(columns=range(0,3)) 

row_marker = 0
for row in table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        df.loc[row_marker,column_marker] = column.get_text()
        column_marker += 1
    row_marker+=1
print("size of dataframe:", df.shape)
df.head(3)

size of dataframe: (287, 3)


,0,1,2
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n


## Clean the data
1. Rename column head
1. Replace "\n" 
1. Filter out Borough = "Not assigned"
1. Replace "Not assigned" neighborhood with name of borough

In [5]:
df.columns = ['Postal Code', 'Borough', 'Nb']                       # Rename column head
df.replace(to_replace='\\n', value="", regex=True, inplace=True)   # Replace \n columns

df=df[df['Borough']!="Not assigned"].dropna()                      # Filter out Borough = "Not assigned"

for r,j in df[df['Nb']=="Not assigned"].iterrows():                # Loop through rows with "Not assigned" neighborhood 
    df.loc[r]['Nb'] = df.loc[r]['Borough']                         # and replace with name of borough

print("size of dataframe:", df.shape)
df.head(3)

size of dataframe: (210, 3)


,Postal Code,Borough,Nb
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


## Group rows with same PostalCode and Borough

In [6]:
df['Neighborhood'] = df.groupby(['Postal Code', 'Borough']).transform(lambda x: ', '.join(sorted(x)))
df_gp = df.groupby(['Postal Code', 'Borough','Neighborhood']).count()
df_gp=df_gp.drop('Nb', axis=1).reset_index()
print("size of dataframe:", df_gp.shape)
df_gp.head()

size of dataframe: (103, 3)


,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
df_gp.shape

(103, 3)